# Recurrent Neural Networks: Obama/Trump Tweet Classifier

For this workshop, you will be training a Recurrent Neural Network to classify between Obama's tweets and Trump's tweets. You will need to be familiar with the theory behind RNNs and/or LSTMs before starting. Visit our lesson [here](http://caisplusplus.usc.edu/blog/curriculum/lesson8) for more info. Only fill in the TODO sections!

---

## Step 1: Load the Data
1. Load in Tweets and corresponding labels (Obama = 0, Trump = 1)
2. Display some sample tweets

In [1]:
TWEETS_DIR = './obama-trump-data/tweets'
LABELS_DIR = './obama-trump-data/labels_np'

######################################
# TODO: specify your own word embeddings file directory here
EMBEDDINGS_DIR = './obama-trump-data/glove.6B.50d.txt'
######################################

In [2]:
import pickle

# Load tweets, labels
tweets = pickle.load(open(TWEETS_DIR,'rb'))
labels = pickle.load(open(LABELS_DIR,'rb'))

# Sample some tweets to display
for i in range(0,100,10):
    print("Tweet: ", tweets[i], ". Label: ", labels[i])

Tweet:  "America should be very proud." President Obama #LoveWins . Label:  0
Tweet:  TO ALL AMERICANS
https://t.co/D7Es6ie4fY . Label:  1
Tweet:  No deal was made last night on DACA. Massive border security would have to be agreed to in exchange for consent. Would be subject to vote. . Label:  1
Tweet:  I was viciously attacked by Mr. Khan at the Democratic Convention. Am I not allowed to respond? Hillary voted for the Iraq war, not me! . Label:  1
Tweet:  .@RudyGiuliani, one of the finest people I know and a former GREAT Mayor of N.Y.C., just took himself out of consideration for "State". . Label:  1
Tweet:  The #TPP establishes the highest labor standards of any trade agreement in history. http://t.co/3vqoancyYp . Label:  0
Tweet:  Thoughts and prayers to the great people of Indiana. You will prevail! . Label:  1
Tweet:  Hillary said she was under sniper fire (while surrounded by USSS.) Turned out to be a total lie. She is not fit to https://t.co/hBIrGj21l6 . Label:  1
Tweet:  The f

## Step 2: Data Preprocessing

Relevant Keras documentation [here](https://keras.io/preprocessing/text/).

1. **Tokenize** the tweets: convert each tweet into a sequence of word indices (Documentation .)
2. **Pad** the input sequences with blank spots at the end to make sure they're all the same length
3. Load in pre-trained word embeddings so that we can convert each word index into a unique **word embedding vector**

**Word embeddings** are a way of encoding words into n-dimensional vectors with continuous values so that the vector contains some information about the words' meanings. **Embedded word vectors** are often more useful than **one-hot vectors** in natural language processing applications, because the vectors themselves contain some embedded information about the word's meaning, instead of just identifying which word is there. 

For example, similar words (e.g. "him" and "her") tend to have similar embedded vectors, whereas if we were just using one-hot vectors (i.e. only one "1" for "him" and one "1" for "her"), no notion of the words' actual meanings would be conveyed. Although this may sound crazy at first, word embeddings can even convey relationships between analogous words: for example: `king-man+woman≈queen`.

For more info on word embeddings, check out this introductory [blog post](https://www.springboard.com/blog/introduction-word-embeddings/). Two of the most common word embeddings methods are [word2vec](https://www.tensorflow.org/tutorials/representation/word2vec) and [GloVe](https://nlp.stanford.edu/projects/glove/).

In [3]:
import numpy as np
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenize the tweets (convert sentence to sequence of words)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets)

sequences = tokenizer.texts_to_sequences(tweets)
word_index = tokenizer.word_index

print('Found %s unique tokens (words).' % len(word_index))

# Pad sequences to ensure samples are the same size
training_data = pad_sequences(sequences)

print("Training data size is (%d,%d)"  % training_data.shape) # shape = (num. tweets, max. length of tweet)
print("Labels are size %d"  % labels.shape)

Using TensorFlow backend.


Found 13049 unique tokens (words).
Training data size is (6136,31)
Labels are size 6136


In [4]:
word_index

{'t': 1,
 'the': 2,
 'co': 3,
 'to': 4,
 'https': 5,
 'and': 6,
 'a': 7,
 'in': 8,
 'of': 9,
 'http': 10,
 'is': 11,
 'president': 12,
 'for': 13,
 'obama': 14,
 'on': 15,
 'you': 16,
 'we': 17,
 'will': 18,
 'rt': 19,
 'at': 20,
 'be': 21,
 'i': 22,
 'our': 23,
 'are': 24,
 'that': 25,
 'this': 26,
 'with': 27,
 'it': 28,
 'have': 29,
 'great': 30,
 'your': 31,
 'more': 32,
 'about': 33,
 'all': 34,
 'not': 35,
 'thank': 36,
 'now': 37,
 'from': 38,
 'people': 39,
 'today': 40,
 'actonclimate': 41,
 'by': 42,
 'time': 43,
 'america': 44,
 'if': 45,
 'out': 46,
 'has': 47,
 'watch': 48,
 'was': 49,
 'just': 50,
 'make': 51,
 'up': 52,
 'my': 53,
 'as': 54,
 'new': 55,
 'get': 56,
 'do': 57,
 'change': 58,
 'they': 59,
 'whitehouse': 60,
 'so': 61,
 'me': 62,
 "it's": 63,
 'jobs': 64,
 'who': 65,
 'no': 66,
 'hillary': 67,
 'american': 68,
 'climate': 69,
 'an': 70,
 'what': 71,
 's': 72,
 'u': 73,
 'live': 74,
 'americans': 75,
 'can': 76,
 'trump': 77,
 'join': 78,
 'big': 79,
 'healt

In [5]:
# Show effect of tokenization, padding
print("Original tweet: ", tweets[0])
print("Tweet after tokenization and padding: ", training_data[0])

Original tweet:  "America should be very proud." President Obama #LoveWins
Tweet after tokenization and padding:  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  44  88  21  83 391  12  14 794]


In [6]:
# Convert words to word embedding vectors

EMBEDDING_DIM = 50
print('Indexing word vectors.')

import os
embeddings_index = {}
f = open(EMBEDDINGS_DIR) # NOTE: if using Windows and getting errors: change this to open(EMBEDDINGS_DIR, 'rb')
for line in f:
    values = line.split()
    word = values[0] # NOTE: if you made that 'rb' change: add to this line, values[0].decode('UTF-8')
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# prepare word embedding matrix
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Indexing word vectors.
Found 400000 word vectors.


In [7]:
# Sample word embedding vector:
print(word_index["computer"]) # retrieve word index
print(embedding_matrix[2645]) # use word index to retrieve word embedding vector

2645
[ 0.079084   -0.81503999  1.79009998  0.91653001  0.10797    -0.55628002
 -0.84426999 -1.49510002  0.13417999  0.63626999  0.35146001  0.25813001
 -0.55028999  0.51055998  0.37408999  0.12092    -1.61660004  0.83653003
  0.14202    -0.52348     0.73452997  0.12207    -0.49079001  0.32532999
  0.45306    -1.58500004 -0.63848001 -1.00530005  0.10454    -0.42984
  3.18099999 -0.62186998  0.16819    -1.01390004  0.064058    0.57844001
 -0.45559999  0.73782998  0.37202999 -0.57722002  0.66441     0.055129
  0.037891    1.32749999  0.30991     0.50696999  1.23570001  0.1274
 -0.11434     0.20709001]


## Step 3: Creating the Classifier Model
Here are some useful resources to help you guys out here:

#### Keras Documentation
* [Embedding layer](https://keras.io/layers/embeddings/) (our word embeddings are pre-trained, so you won't have to worry too much about this)
* [**Recurrent layers (e.g. RNN, LSTM)**](https://keras.io/layers/recurrent/) (Feel free to play around with different layers.)
* [Dense layer](https://keras.io/layers/core/#dense) (Use this for your final classification "vote")
* [Activation functions](https://keras.io/activations/)
* [Dropout](https://keras.io/layers/core/#dropout), [Batch Normalization](https://keras.io/layers/normalization/)
* [Optimizers](https://keras.io/optimizers/)

#### Examples
* [Keras sequential model guide](https://keras.io/getting-started/sequential-model-guide/) (Scroll down to "Sequence classification with LSTM")
* [Keras LSTM for IMDB review sentiment analysis](https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py)

**Hint**: if you have multiple recurrent layers, remember to use `return_sequences=True` if and only if you're adding another recurrent layer after the current one. This makes it so that the recurrent layer spits out an output after each timestep (or element in the sequence), instead of just at the very end of the sequence.

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import metrics
from keras.models import Model
from keras import optimizers
import pickle

In [25]:
timesteps = training_data.shape[1]

model = Sequential()

# Add pre-trained embedding layer 
    # converts word indices to GloVe word embedding vectors as they're fed in
model.add(Embedding(len(word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=training_data.shape[1],
                    trainable=False))

# At this point, each individual training sample is now a sequence of word embedding vectors

######################################
# TODO: define the rest of the network!
model.add(LSTM(32, return_sequences=True, input_shape=(timesteps, EMBEDDING_DIM)))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=True, input_shape=(timesteps, EMBEDDING_DIM)))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False, input_shape=(timesteps, EMBEDDING_DIM)))
model.add(Dense(1))
model.add(Activation('sigmoid'))

######################################

print(model.summary())


######################################
# TODO: pick out a loss function and corresponding optimizer.
    # Hint: there's one "correct" loss function we have in mind,
        # since we're just classifying between 0 and 1.
    # You can pick your own optimizer, but Keras's documentation recommends one for RNNs 

LOSS = 'binary_crossentropy'
OPTIMIZER = optimizers.Adam()
######################################


model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 31, 50)            652500    
_________________________________________________________________
lstm_22 (LSTM)               (None, 31, 32)            10624     
_________________________________________________________________
dropout_15 (Dropout)         (None, 31, 32)            0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 31, 32)            8320      
_________________________________________________________________
dropout_16 (Dropout)         (None, 31, 32)            0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
__________

## 3. Train/Evaluate the Model
Train the network. Look to make sure that the loss is decreasing and the accuracy is decreasing, but be on the lookout for overfitting. 

Aim for **90%** final validation accuracy! (Pretty much the same as using a separate test set)

In [ ]:
#####################################
# TODO: pick number of epochs and batch size

EPOCHS = 10
BATCH_SIZE = 32
#####################################

model.fit(training_data, labels, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split =.2)

Train on 4908 samples, validate on 1228 samples
Epoch 1/10
4908/4908 [==============================] - 14s 3ms/step - loss: 0.4782 - binary_accuracy: 0.7586 - val_loss: 0.3413 - val_binary_accuracy: 0.8485
Epoch 2/10
4908/4908 [==============================] - 10s 2ms/step - loss: 0.2549 - binary_accuracy: 0.8969 - val_loss: 0.2195 - val_binary_accuracy: 0.9145
Epoch 3/10
4908/4908 [==============================] - 10s 2ms/step - loss: 0.1944 - binary_accuracy: 0.9242 - val_loss: 0.1886 - val_binary_accuracy: 0.9226
Epoch 4/10
4908/4908 [==============================] - 11s 2ms/step - loss: 0.1638 - binary_accuracy: 0.9385 - val_loss: 0.1811 - val_binary_accuracy: 0.9283
Epoch 5/10
4908/4908 [==============================] - 11s 2ms/step - loss: 0.1452 - binary_accuracy: 0.9430 - val_loss: 0.1500 - val_binary_accuracy: 0.9430
Epoch 6/10
4908/4908 [==============================] - 11s 2ms/step - loss: 0.1239 - binary_accuracy: 0.9550 - val_loss: 0.1471 - val_binary_accuracy: 0.942